### CÓDIGO COMÚN

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, MinMaxScaler
import statistics as stat

print('Librerías importadas correctamente')

Librerías importadas correctamente


In [2]:
bmw = pd.read_csv('/Users/davidlarre/Desktop/PROYECTOS/M10 - ENTREGABLES/ENTREGABLE 1/data/bmw_pricing.csv')
df_bmw = bmw.copy()

In [3]:
# Se modifica el dtype de las columnas 'fecha_registro' y 'fecha_venta' a datetime
for i in ['fecha_registro','fecha_venta']:
    df_bmw[i]=pd.to_datetime(df_bmw[i])
# Comprobamos el cambio de dtype
print(df_bmw[['fecha_registro','fecha_venta']].dtypes)

fecha_registro    datetime64[ns]
fecha_venta       datetime64[ns]
dtype: object


In [4]:
# Consideramos que las fechas completas no aportan información relevante, por lo que vamos a crear nuevas columnas solo con el año
df_bmw['year_registro'] = df_bmw['fecha_registro'].dt.year
df_bmw['year_venta'] = df_bmw['fecha_venta'].dt.year
# Comprobamos que se han creado correctamente
df_bmw[['year_registro','year_venta']].head(10)

,year_registro,year_venta
0,2012.0,2018.0
1,NaN,2018.0
2,2012.0,2018.0
3,NaN,2018.0
4,NaN,2018.0
5,2011.0,2018.0
6,NaN,2018.0
7,2009.0,2018.0
8,NaN,2018.0
9,NaN,2018.0


In [5]:
# Creamos una función para crear un DataFrame con la cantidad de nulos, el porcentaje de nulos y los valores únicos de cada columna. Ordenaremos el df por el porcentaje de nulos
def tabla_nulos(dataset):
    nulos = dataset.isnull().sum()
    porcentaje = dataset.isnull().mean() * 100
    unicos = dataset.nunique()
    
    df_nulos = pd.DataFrame({
        'NULOS': nulos,
        '% NULOS': porcentaje,
        'VALORES UNICOS': unicos
        }).sort_values(by='% NULOS', ascending=False)
    return df_nulos
    
tabla_nulos(df_bmw)

,NULOS,% NULOS,VALORES UNICOS
asientos_traseros_plegables,3391,70.018584,2
year_registro,2423,50.030973,23
fecha_registro,2423,50.030973,180
tipo_coche,1460,30.146603,8
marca,970,20.028908,1
bluetooth,728,15.032005,2
alerta_lim_velocidad,728,15.032005,2
aire_acondicionado,486,10.035102,2
color,445,9.188520,10
precio,6,0.123890,437


In [6]:
# Creamos un diccionario para almacenar el porcentaje que tiene cada valor respecto al total de cada columna
distribuciones = {}
for i in df_bmw:
    distribuciones[i] = df_bmw[i].value_counts(normalize=True)

## COMPROBACION DE LA IMPUTACIÓN DE TIPO COCHE

### VERSIÓN 1 - 72%

In [7]:
tabla_nulos(bmw).loc[['tipo_coche']]

,NULOS,% NULOS,VALORES UNICOS
tipo_coche,1460,30.146603,8


In [8]:
df_bmw['tipo_coche'].unique()

array([nan, 'convertible', 'coupe', 'estate', 'hatchback', 'sedan',
       'subcompact', 'suv', 'van'], dtype=object)

In [9]:
def obtener_moda(x):
    moda = x.mode()
    if not moda.empty:
        return moda.iloc[0]
    else:
        return np.nan

In [10]:
df_bmw

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta,year_registro,year_venta
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,False,NaN,True,NaN,True,NaN,11300.0,2018-01-01,2012.0,2018.0
1,BMW,M4,13929.0,317.0,NaT,petrol,grey,convertible,True,True,False,NaN,False,True,True,True,69700.0,2018-02-01,NaN,2018.0
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,NaN,False,False,False,NaN,True,False,True,False,10200.0,2018-02-01,2012.0,2018.0
3,BMW,420,128035.0,135.0,NaT,diesel,red,convertible,True,True,False,NaN,True,True,True,NaN,25100.0,2018-02-01,NaN,2018.0
4,BMW,425,97097.0,160.0,NaT,diesel,silver,NaN,True,True,False,False,False,True,True,True,33400.0,2018-04-01,NaN,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4838,BMW,218 Gran Tourer,39743.0,110.0,NaT,diesel,black,NaN,False,True,False,NaN,False,False,True,False,14600.0,2018-08-01,NaN,2018.0
4839,BMW,218 Active Tourer,49832.0,100.0,2015-06-01,diesel,grey,NaN,False,True,False,NaN,False,False,True,True,17500.0,2018-08-01,2015.0,2018.0
4840,BMW,218 Gran Tourer,19633.0,110.0,2015-10-01,diesel,grey,van,False,True,False,NaN,False,False,True,True,17000.0,2018-09-01,2015.0,2018.0
4841,BMW,218 Active Tourer,27920.0,110.0,2016-04-01,diesel,brown,van,True,True,False,False,False,False,True,True,22700.0,2018-09-01,2016.0,2018.0


In [11]:
df_bmw['edad_coche'] = df_bmw['year_venta'] - df_bmw['year_registro']

In [12]:
'''COLUMNS_1 = ['modelo', 'potencia', 'year_registro']
df_nulls_1 = df_bmw.groupby(COLUMNS_1)['tipo_coche'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).reset_index()
df_nulls_1 = df_nulls_1.dropna(subset=['tipo_coche'])
df_nulls_1.rename(columns={"tipo_coche":"tipo_coche_1"},inplace=True)

COLUMNS_2 = ['modelo', 'potencia']
df_nulls_2 = df_bmw.groupby(COLUMNS_2)['tipo_coche'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).reset_index()
df_nulls_2 = df_nulls_2.dropna(subset=['tipo_coche'])
df_nulls_2.rename(columns={"tipo_coche":"tipo_coche_2"},inplace=True)

COLUMNS_3 = ['modelo']
df_nulls_3 = df_bmw.groupby(COLUMNS_3)['tipo_coche'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).reset_index()
df_nulls_3 = df_nulls_3.dropna(subset=['tipo_coche'])
df_nulls_3.rename(columns={"tipo_coche":"tipo_coche_3"},inplace=True)'''

'COLUMNS_1 = [\'modelo\', \'potencia\', \'year_registro\']\ndf_nulls_1 = df_bmw.groupby(COLUMNS_1)[\'tipo_coche\'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).reset_index()\ndf_nulls_1 = df_nulls_1.dropna(subset=[\'tipo_coche\'])\ndf_nulls_1.rename(columns={"tipo_coche":"tipo_coche_1"},inplace=True)\n\nCOLUMNS_2 = [\'modelo\', \'potencia\']\ndf_nulls_2 = df_bmw.groupby(COLUMNS_2)[\'tipo_coche\'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).reset_index()\ndf_nulls_2 = df_nulls_2.dropna(subset=[\'tipo_coche\'])\ndf_nulls_2.rename(columns={"tipo_coche":"tipo_coche_2"},inplace=True)\n\nCOLUMNS_3 = [\'modelo\']\ndf_nulls_3 = df_bmw.groupby(COLUMNS_3)[\'tipo_coche\'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).reset_index()\ndf_nulls_3 = df_nulls_3.dropna(subset=[\'tipo_coche\'])\ndf_nulls_3.rename(columns={"tipo_coche":"tipo_coche_3"},inplace=True)'

In [13]:
'''df_bmw = df_bmw.merge(df_nulls_1,how='left',left_on=COLUMNS_1,right_on=COLUMNS_1)
df_bmw = df_bmw.merge(df_nulls_2,how='left',left_on=COLUMNS_2,right_on=COLUMNS_2)
df_bmw = df_bmw.merge(df_nulls_3,how='left',left_on=COLUMNS_3,right_on=COLUMNS_3)

df_bmw['tipo_coche'] = df_bmw[['tipo_coche','tipo_coche_1','tipo_coche_2','tipo_coche_3']].bfill(axis=1).iloc[:, 0]

df_bmw.drop(columns=['tipo_coche_1','tipo_coche_2','tipo_coche_3'],inplace=True)'''

"df_bmw = df_bmw.merge(df_nulls_1,how='left',left_on=COLUMNS_1,right_on=COLUMNS_1)\ndf_bmw = df_bmw.merge(df_nulls_2,how='left',left_on=COLUMNS_2,right_on=COLUMNS_2)\ndf_bmw = df_bmw.merge(df_nulls_3,how='left',left_on=COLUMNS_3,right_on=COLUMNS_3)\n\ndf_bmw['tipo_coche'] = df_bmw[['tipo_coche','tipo_coche_1','tipo_coche_2','tipo_coche_3']].bfill(axis=1).iloc[:, 0]\n\ndf_bmw.drop(columns=['tipo_coche_1','tipo_coche_2','tipo_coche_3'],inplace=True)"

In [14]:
'''moda_paso_1 = df_bmw.groupby(['modelo', 'potencia', 'year_registro'])['tipo_coche'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'tipo_coche': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo', 'potencia', 'year_registro'], how='left')
df_bmw['tipo_coche'] = df_bmw['tipo_coche'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)
tabla_nulos(df_bmw).loc[['tipo_coche']]
moda_paso_2 = df_bmw.groupby(['modelo', 'potencia'])['tipo_coche'].agg(obtener_moda).reset_index()
moda_paso_2 = moda_paso_2.rename(columns={'tipo_coche': 'MODA_PASO_2'})
df_bmw = df_bmw.merge(moda_paso_2, on=['modelo', 'potencia'], how='left')
df_bmw['tipo_coche'] = df_bmw['tipo_coche'].fillna(df_bmw['MODA_PASO_2'])
df_bmw.drop(columns=['MODA_PASO_2'], inplace=True)
tabla_nulos(df_bmw).loc[['tipo_coche']]
moda_paso_2 = df_bmw.groupby(['modelo'])['tipo_coche'].agg(obtener_moda).reset_index()
moda_paso_2 = moda_paso_2.rename(columns={'tipo_coche': 'MODA_PASO_2'})
df_bmw = df_bmw.merge(moda_paso_2, on=['modelo'], how='left')
df_bmw['tipo_coche'] = df_bmw['tipo_coche'].fillna(df_bmw['MODA_PASO_2'])
df_bmw.drop(columns=['MODA_PASO_2'], inplace=True)
moda_general = df_bmw['tipo_coche'].mode().iloc[0]
df_bmw['tipo_coche'] = df_bmw['tipo_coche'].fillna(moda_general)
tabla_nulos(df_bmw).loc[['tipo_coche']]
'''

"moda_paso_1 = df_bmw.groupby(['modelo', 'potencia', 'year_registro'])['tipo_coche'].agg(obtener_moda).reset_index()\nmoda_paso_1 = moda_paso_1.rename(columns={'tipo_coche': 'MODA_PASO_1'})\ndf_bmw = df_bmw.merge(moda_paso_1, on=['modelo', 'potencia', 'year_registro'], how='left')\ndf_bmw['tipo_coche'] = df_bmw['tipo_coche'].fillna(df_bmw['MODA_PASO_1'])\ndf_bmw.drop(columns=['MODA_PASO_1'], inplace=True)\ntabla_nulos(df_bmw).loc[['tipo_coche']]\nmoda_paso_2 = df_bmw.groupby(['modelo', 'potencia'])['tipo_coche'].agg(obtener_moda).reset_index()\nmoda_paso_2 = moda_paso_2.rename(columns={'tipo_coche': 'MODA_PASO_2'})\ndf_bmw = df_bmw.merge(moda_paso_2, on=['modelo', 'potencia'], how='left')\ndf_bmw['tipo_coche'] = df_bmw['tipo_coche'].fillna(df_bmw['MODA_PASO_2'])\ndf_bmw.drop(columns=['MODA_PASO_2'], inplace=True)\ntabla_nulos(df_bmw).loc[['tipo_coche']]\nmoda_paso_2 = df_bmw.groupby(['modelo'])['tipo_coche'].agg(obtener_moda).reset_index()\nmoda_paso_2 = moda_paso_2.rename(columns={'ti

In [15]:
# 1. Cargar el dataset original
df_bmw_original = pd.read_csv('/Users/davidlarre/Desktop/PROYECTOS/M10 - ENTREGABLES/ENTREGABLE 1/data/bmw_original.csv')

# 2. Asegurarnos de que ambos dataframes tienen el mismo índice (o una columna común para alinear)
# Puedes hacer merge si hay un ID único, pero si no, asumimos que el orden es el mismo

# 3. Identificar las filas que originalmente tenían nulos en 'tipo_coche'
mask_nulos_originales = bmw['tipo_coche'].isnull()

# 4. Filtrar solo esas filas para comparar
tipo_coche_imputado = df_bmw.loc[mask_nulos_originales, 'tipo_coche']
tipo_coche_real = df_bmw_original.loc[mask_nulos_originales, 'tipo_coche']

# 5. Calcular aciertos
aciertos = (tipo_coche_imputado == tipo_coche_real).sum()
total = mask_nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 6. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 0 de 1460 imputaciones
Porcentaje de acierto: 0.00%


### VERSIÓN 2 - MODELO

In [16]:
'''import pandas as pd
import statistics as stat

# Mostrar info inicial
bmw.info()

# Asumo que tienes definida esta función para ver nulos:
def get_tabla_nulos(df):
    return pd.DataFrame({'nulos': df.isnull().sum(), 'porcentaje': 100*df.isnull().mean()})

print(get_tabla_nulos(bmw))

# Distribución original de tipo_coche
print(bmw['tipo_coche'].value_counts(normalize=True))

# Mostrar distribución agrupada (puedes descomentar si quieres ver todo)
# pd.set_option('display.max_rows', None)
# print(bmw.groupby(['modelo'])['tipo_coche'].value_counts(normalize=True))
# print(bmw.groupby(['modelo','potencia'])['tipo_coche'].value_counts(normalize=True))
# pd.set_option('display.max_rows', 20)

# Calcular la moda de tipo_coche por modelo y potencia, y solo por modelo
tipo_coche_mode_table = bmw[~bmw['tipo_coche'].isnull()].groupby(['modelo','potencia'])['tipo_coche'].agg(lambda x: stat.mode(x))
tipo_coche_mode_por_modelo = bmw[~bmw['tipo_coche'].isnull()].groupby(['modelo'])['tipo_coche'].agg(lambda x: stat.mode(x))

print(f"Modelos únicos: {len(bmw['modelo'].unique())}")
print(f"Modas por modelo: {len(tipo_coche_mode_por_modelo)}")

# Imputar nulos con moda por (modelo,potencia), si no existe con moda solo por modelo, sino con moda global
for i in bmw[bmw['tipo_coche'].isnull()].index:
    modelo = bmw.loc[i, 'modelo']
    potencia = bmw.loc[i, 'potencia']
    if (modelo, potencia) in tipo_coche_mode_table.index:
        tipo_coche = tipo_coche_mode_table.loc[(modelo, potencia)]
    elif modelo in tipo_coche_mode_por_modelo.index:
        tipo_coche = tipo_coche_mode_por_modelo.loc[modelo]
    else:
        tipo_coche = bmw['tipo_coche'].mode().values[0]
    bmw.loc[i, 'tipo_coche'] = tipo_coche

print(get_tabla_nulos(bmw))

print("Distribución después de imputar:")
print(bmw['tipo_coche'].value_counts(normalize=True))

# Ahora, para medir el acierto con el dataset original

# 1. Cargar el dataset original
df_bmw_original = pd.read_csv('/Users/davidlarre/Desktop/PROYECTOS/ENTREGABLE 1/bmw_original.csv')

# 2. Crear máscara con las filas que originalmente tenían nulos
mask_nulos_originales = bmw['tipo_coche'].isnull()

# 3. Filtrar solo esas filas para comparar
tipo_coche_imputado = df_bmw.loc[mask_nulos_originales, 'tipo_coche']
tipo_coche_real = df_bmw_original.loc[mask_nulos_originales, 'tipo_coche']

# 4. Calcular aciertos
aciertos = (tipo_coche_imputado == tipo_coche_real).sum()
total = mask_nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")'''

'import pandas as pd\nimport statistics as stat\n\n# Mostrar info inicial\nbmw.info()\n\n# Asumo que tienes definida esta función para ver nulos:\ndef get_tabla_nulos(df):\n    return pd.DataFrame({\'nulos\': df.isnull().sum(), \'porcentaje\': 100*df.isnull().mean()})\n\nprint(get_tabla_nulos(bmw))\n\n# Distribución original de tipo_coche\nprint(bmw[\'tipo_coche\'].value_counts(normalize=True))\n\n# Mostrar distribución agrupada (puedes descomentar si quieres ver todo)\n# pd.set_option(\'display.max_rows\', None)\n# print(bmw.groupby([\'modelo\'])[\'tipo_coche\'].value_counts(normalize=True))\n# print(bmw.groupby([\'modelo\',\'potencia\'])[\'tipo_coche\'].value_counts(normalize=True))\n# pd.set_option(\'display.max_rows\', 20)\n\n# Calcular la moda de tipo_coche por modelo y potencia, y solo por modelo\ntipo_coche_mode_table = bmw[~bmw[\'tipo_coche\'].isnull()].groupby([\'modelo\',\'potencia\'])[\'tipo_coche\'].agg(lambda x: stat.mode(x))\ntipo_coche_mode_por_modelo = bmw[~bmw[\'tipo_c

# VERSIÓN 3

In [17]:
modelo_tipo_coche = {
    # Hatchbacks / Subcompactos
    '114': 'hatchback',
    '116': 'hatchback',
    '118': 'hatchback',
    '120': 'hatchback',
    '123': 'hatchback',
    '125': 'hatchback',
    '135': 'hatchback',
    'M135': 'hatchback',

    # Coupé
    '218': 'coupe',
    '220': 'coupe',
    '225': 'coupe',
    '228': 'coupe',  # Si aparece
    '230': 'coupe',  # Si aparece
    'M235': 'coupe',
    'M240': 'coupe',  # Si aparece
    '420': 'coupe',
    '425': 'coupe',
    '430': 'coupe',
    '435': 'coupe',
    '440': 'coupe',  # Si aparece
    'M4': 'coupe',

    # Coupé 4 puertas (clasificados como coupe también)
    '640 Gran Coupé': 'coupe',
    '435 Gran Coupé': 'coupe',
    '430 Gran Coupé': 'coupe',
    '420 Gran Coupé': 'coupe',
    '418 Gran Coupé': 'coupe',

    # Berlina / Sedán
    '316': 'sedan',
    '318': 'sedan',
    '320': 'sedan',
    '325': 'sedan',
    '328': 'sedan',
    '330': 'sedan',
    '335': 'sedan',
    'M3': 'sedan',
    '520': 'sedan',
    '523': 'sedan',
    '525': 'sedan',
    '528': 'sedan',
    '530': 'sedan',
    '535': 'sedan',
    'M5': 'sedan',
    '518': 'sedan',
    'ActiveHybrid 5': 'sedan',
    'M550': 'sedan',
    '730': 'sedan',
    '735': 'sedan',
    '740': 'sedan',
    '750': 'sedan',

    # Gran Turismo (más cercanos a hatchback grande / fastback)
    '320 Gran Turismo': 'hatchback',
    '325 Gran Turismo': 'hatchback',
    '330 Gran Turismo': 'hatchback',
    '335 Gran Turismo': 'hatchback',
    '520 Gran Turismo': 'hatchback',
    '530 Gran Turismo': 'hatchback',
    '535 Gran Turismo': 'hatchback',
    '318 Gran Turismo': 'hatchback',

    # SUV
    'X1': 'suv',
    'X3': 'suv',
    'X4': 'suv',
    'X5': 'suv',
    'X5 M': 'suv',
    'X5 M50': 'suv',
    'X6': 'suv',
    'X6 M': 'suv',

    # Monovolúmenes / Van
    '218 Active Tourer': 'van',
    '216 Active Tourer': 'van',
    '220 Active Tourer': 'van',
    '225 Active Tourer': 'van',
    'Active Tourer': 'van',
    '218 Gran Tourer': 'van',
    '216 Gran Tourer': 'van',
    '214 Gran Tourer': 'van',

    # Convertible
    'Z4': 'convertible',
    'i8': 'convertible',  # Híbrido deportivo tipo roadster

    # Eléctrico compacto
    'i3': 'subcompact',

    # Otros posibles
    '630': 'coupe',
    '635': 'coupe',
    '640': 'coupe',
}

In [18]:
df_bmw['tipo_coche_imputado'] = df_bmw['modelo'].map(modelo_tipo_coche)

In [19]:
df_bmw['tipo_coche'] = df_bmw['tipo_coche'].fillna(df_bmw['tipo_coche_imputado'])

In [20]:
# 1. Cargar el dataset original
df_bmw_original = pd.read_csv('/Users/davidlarre/Desktop/PROYECTOS/M10 - ENTREGABLES/ENTREGABLE 1/data/bmw_original.csv')

# 2. Asegurarnos de que ambos dataframes tienen el mismo índice (o una columna común para alinear)
# Puedes hacer merge si hay un ID único, pero si no, asumimos que el orden es el mismo

# 3. Identificar las filas que originalmente tenían nulos en 'tipo_coche'
mask_nulos_originales = bmw['tipo_coche'].isnull()

# 4. Filtrar solo esas filas para comparar
tipo_coche_imputado = df_bmw.loc[mask_nulos_originales, 'tipo_coche']
tipo_coche_real = df_bmw_original.loc[mask_nulos_originales, 'tipo_coche']

# 5. Calcular aciertos
aciertos = (tipo_coche_imputado == tipo_coche_real).sum()
total = mask_nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 6. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 835 de 1460 imputaciones
Porcentaje de acierto: 57.19%


## COMPROBACION DE LA IMPUTACIÓN COLOR

### VERSIÓN 1 - 30% ACIERTO

In [21]:
''' import numpy as np

# 1. Imputar primero por moda de color por modelo
moda_color_modelo = df_bmw.groupby('modelo')['color'].agg(obtener_moda).reset_index()
moda_color_modelo = moda_color_modelo.rename(columns={'color': 'MODA_COLOR_MODELO'})
df_bmw = df_bmw.merge(moda_color_modelo, on='modelo', how='left')
df_bmw['color'] = df_bmw['color'].fillna(df_bmw['MODA_COLOR_MODELO'])
df_bmw.drop(columns=['MODA_COLOR_MODELO'], inplace=True)

# 2. Imputar lo que quede nulo según la distribución general
# (Preserva la distribución original de los colores)
distribucion_colores = df_bmw['color'].value_counts(normalize=True)
mask_nulos_color = df_bmw['color'].isnull()
n_nulos = mask_nulos_color.sum()

if n_nulos > 0:
    np.random.seed(42)  # Para reproducibilidad
    colores_random = np.random.choice(
        distribucion_colores.index,
        size=n_nulos,
        p=distribucion_colores.values
    )
    df_bmw.loc[mask_nulos_color, 'color'] = colores_random

# 3. Verificamos que no queden nulos
tabla_nulos(df_bmw).loc[['color']]'''

" import numpy as np\n\n# 1. Imputar primero por moda de color por modelo\nmoda_color_modelo = df_bmw.groupby('modelo')['color'].agg(obtener_moda).reset_index()\nmoda_color_modelo = moda_color_modelo.rename(columns={'color': 'MODA_COLOR_MODELO'})\ndf_bmw = df_bmw.merge(moda_color_modelo, on='modelo', how='left')\ndf_bmw['color'] = df_bmw['color'].fillna(df_bmw['MODA_COLOR_MODELO'])\ndf_bmw.drop(columns=['MODA_COLOR_MODELO'], inplace=True)\n\n# 2. Imputar lo que quede nulo según la distribución general\n# (Preserva la distribución original de los colores)\ndistribucion_colores = df_bmw['color'].value_counts(normalize=True)\nmask_nulos_color = df_bmw['color'].isnull()\nn_nulos = mask_nulos_color.sum()\n\nif n_nulos > 0:\n    np.random.seed(42)  # Para reproducibilidad\n    colores_random = np.random.choice(\n        distribucion_colores.index,\n        size=n_nulos,\n        p=distribucion_colores.values\n    )\n    df_bmw.loc[mask_nulos_color, 'color'] = colores_random\n\n# 3. Verificam

In [22]:
# 3. Identificar las filas que originalmente tenían nulos en 'color'
mask_nulos_originales = bmw['color'].isnull()

# 4. Filtrar solo esas filas para comparar
color_imputado = df_bmw.loc[mask_nulos_originales, 'color']
color_real = df_bmw_original.loc[mask_nulos_originales, 'color']

# 5. Calcular aciertos
aciertos = (color_imputado == color_real).sum()
total = mask_nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 6. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 0 de 445 imputaciones
Porcentaje de acierto: 0.00%


### VERSIÓN 2 - 32% ACIERTO

In [23]:
'''def obtener_moda(x):
    return x.mode().iloc[0] if not x.mode().empty else np.nan

moda_color_1 = df_bmw.groupby(['modelo', 'tipo_coche'])['color'].agg(obtener_moda).reset_index()
moda_color_1.rename(columns={'color': 'MODA_COLOR_1'}, inplace=True)
df_bmw = df_bmw.merge(moda_color_1, on=['modelo', 'tipo_coche'], how='left')
df_bmw['color'] = df_bmw['color'].fillna(df_bmw['MODA_COLOR_1'])
df_bmw.drop(columns=['MODA_COLOR_1'], inplace=True)

moda_color_2 = df_bmw.groupby(['modelo'])['color'].agg(obtener_moda).reset_index()
moda_color_2.rename(columns={'color': 'MODA_COLOR_2'}, inplace=True)
df_bmw = df_bmw.merge(moda_color_2, on='modelo', how='left')
df_bmw['color'] = df_bmw['color'].fillna(df_bmw['MODA_COLOR_2'])
df_bmw.drop(columns=['MODA_COLOR_2'], inplace=True)

moda_color_3 = df_bmw.groupby(['tipo_coche'])['color'].agg(obtener_moda).reset_index()
moda_color_3.rename(columns={'color': 'MODA_COLOR_3'}, inplace=True)
df_bmw = df_bmw.merge(moda_color_3, on='tipo_coche', how='left')
df_bmw['color'] = df_bmw['color'].fillna(df_bmw['MODA_COLOR_3'])
df_bmw.drop(columns=['MODA_COLOR_3'], inplace=True)

moda_general_color = df_bmw['color'].mode().iloc[0]
df_bmw['color'] = df_bmw['color'].fillna(moda_general_color)

tabla_nulos(df_bmw).loc[['color']]
df_bmw['color'].value_counts(normalize=True)'''

"def obtener_moda(x):\n    return x.mode().iloc[0] if not x.mode().empty else np.nan\n\nmoda_color_1 = df_bmw.groupby(['modelo', 'tipo_coche'])['color'].agg(obtener_moda).reset_index()\nmoda_color_1.rename(columns={'color': 'MODA_COLOR_1'}, inplace=True)\ndf_bmw = df_bmw.merge(moda_color_1, on=['modelo', 'tipo_coche'], how='left')\ndf_bmw['color'] = df_bmw['color'].fillna(df_bmw['MODA_COLOR_1'])\ndf_bmw.drop(columns=['MODA_COLOR_1'], inplace=True)\n\nmoda_color_2 = df_bmw.groupby(['modelo'])['color'].agg(obtener_moda).reset_index()\nmoda_color_2.rename(columns={'color': 'MODA_COLOR_2'}, inplace=True)\ndf_bmw = df_bmw.merge(moda_color_2, on='modelo', how='left')\ndf_bmw['color'] = df_bmw['color'].fillna(df_bmw['MODA_COLOR_2'])\ndf_bmw.drop(columns=['MODA_COLOR_2'], inplace=True)\n\nmoda_color_3 = df_bmw.groupby(['tipo_coche'])['color'].agg(obtener_moda).reset_index()\nmoda_color_3.rename(columns={'color': 'MODA_COLOR_3'}, inplace=True)\ndf_bmw = df_bmw.merge(moda_color_3, on='tipo_coche

In [24]:
# 3. Identificar las filas que originalmente tenían nulos en 'color'
mask_nulos_originales = bmw['color'].isnull()

# 4. Filtrar solo esas filas para comparar
color_imputado = df_bmw.loc[mask_nulos_originales, 'color']
color_real = df_bmw_original.loc[mask_nulos_originales, 'color']

# 5. Calcular aciertos
aciertos = (color_imputado == color_real).sum()
total = mask_nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 6. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 0 de 445 imputaciones
Porcentaje de acierto: 0.00%


### VERSIÓN 3 - 80% ACIERTO

In [25]:
def obtener_color_binario(x):
  if str(x) == 'nan':
    return np.nan
  elif x in ['black','grey','blue','white']:
    return 1
  else:
    return 0

In [26]:
df_bmw['color_estandar'] = df_bmw['color'].map(obtener_color_binario)

In [27]:
df_bmw_original['color_estandar'] = df_bmw_original['color'].map(obtener_color_binario)

In [28]:
df_bmw[['color','color_estandar']]

,color,color_estandar
0,black,1.0
1,grey,1.0
2,white,1.0
3,red,0.0
4,silver,0.0
...,...,...
4838,black,1.0
4839,grey,1.0
4840,grey,1.0
4841,brown,0.0


In [29]:
# guardemos los valores de la moda para color_estandar en tablas diferentes por niveles de agrupación
tabla_moda_color_estandar = df_bmw[~(df_bmw['color_estandar'].isnull())].groupby(['modelo','tipo_coche','potencia'])['color_estandar'].agg([stat.mode])

In [30]:
tabla_moda_color_estandar_por_modelo_y_tipo_coche = df_bmw[~(df_bmw['color_estandar'].isnull())].groupby(['modelo','tipo_coche'])['color_estandar'].agg([stat.mode])

In [31]:
tabla_moda_color_estandar_por_modelo = df_bmw[~(df_bmw['color_estandar'].isnull())].groupby(['modelo'])['color_estandar'].agg([stat.mode])

In [32]:
tabla_moda_color_estandar_por_tipo_coche = df_bmw[~(df_bmw['color_estandar'].isnull())].groupby(['tipo_coche'])['color_estandar'].agg([stat.mode])

In [33]:
len(tabla_moda_color_estandar_por_modelo)==len(df_bmw['modelo'].unique())

False

In [34]:
#falta un modelo que es unico en todo el dataset y tiene color_estandar nulo
len(df_bmw['modelo'].unique())

77

In [35]:
# reemplacemoslos nulos en color_estandar con la moda por modelo, tipo_coche y potencia. Cuando no es posible usemos la moda solo por modelo y tipo_coche. Si tampoco esta existe
# usemos la moda por modelo, e si tampoco esta existe le ponemos la moda de color_estandar para tipo_coche, osea 1

for i in df_bmw[df_bmw['color_estandar'].isnull()].index:
  modelo = df_bmw.loc[i,'modelo']
  tipo_coche = df_bmw.loc[i,'tipo_coche']
  potencia = df_bmw.loc[i,'potencia']
 # existe la moda por modelo, tipo_coche y potencia?
  if (modelo,tipo_coche,potencia) in tabla_moda_color_estandar.index:
    color_estandar = tabla_moda_color_estandar.loc[(modelo,tipo_coche,potencia)].values[0]
 # si no, existe la moda solo por modelo y tipo_coche?
  elif (modelo,tipo_coche) in tabla_moda_color_estandar_por_modelo_y_tipo_coche.index:
    color_estandar = tabla_moda_color_estandar_por_modelo_y_tipo_coche.loc[(modelo,tipo_coche)].values[0]
 # si no, existe la moda solo por modelo?
  elif (modelo) in tabla_moda_color_estandar_por_modelo.index:
    color_estandar = tabla_moda_color_estandar_por_modelo.loc[(modelo)].values[0]
  else:
    color_estandar = tabla_moda_color_estandar_por_tipo_coche.loc[(tipo_coche)].values[0]
  df_bmw.loc[i,'color_estandar'] = color_estandar

In [36]:
tabla_nulos(df_bmw)

,NULOS,% NULOS,VALORES UNICOS
asientos_traseros_plegables,3391,70.018584,2
fecha_registro,2423,50.030973,180
edad_coche,2423,50.030973,25
year_registro,2423,50.030973,23
marca,970,20.028908,1
bluetooth,728,15.032005,2
alerta_lim_velocidad,728,15.032005,2
aire_acondicionado,486,10.035102,2
color,445,9.188520,10
tipo_coche_imputado,7,0.144539,7


In [37]:
df_bmw_original['color_estandar'].value_counts(normalize=True)

color_estandar
1    0.837497
0    0.162503
Name: proportion, dtype: float64

In [38]:
df_bmw.head()

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,...,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta,year_registro,year_venta,edad_coche,tipo_coche_imputado,color_estandar
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,hatchback,True,True,...,NaN,True,NaN,11300.0,2018-01-01,2012.0,2018.0,6.0,hatchback,1.0
1,BMW,M4,13929.0,317.0,NaT,petrol,grey,convertible,True,True,...,True,True,True,69700.0,2018-02-01,NaN,2018.0,NaN,coupe,1.0
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,sedan,False,False,...,False,True,False,10200.0,2018-02-01,2012.0,2018.0,6.0,sedan,1.0
3,BMW,420,128035.0,135.0,NaT,diesel,red,convertible,True,True,...,True,True,NaN,25100.0,2018-02-01,NaN,2018.0,NaN,coupe,0.0
4,BMW,425,97097.0,160.0,NaT,diesel,silver,coupe,True,True,...,True,True,True,33400.0,2018-04-01,NaN,2018.0,NaN,coupe,0.0


In [39]:
# despues de la sostitucion de los nulos se ha mantenido la distribucion inicial de los valores ----> OK

In [40]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = df_bmw['color'].isnull()

# 2. Filtrar solo esas filas para comparar
color_imputado = df_bmw.loc[nulos_originales, 'color_estandar'] # df donde se han imputado nulos
color_real = df_bmw_original.loc[nulos_originales, 'color_estandar'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (color_imputado == color_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 355 de 445 imputaciones
Porcentaje de acierto: 79.78%


## COMPROBACIÓN DE LA IMPUTACIÓN EDAD COCHE

### VERSIÓN 1 - 42%

In [41]:
df_bmw_original = pd.read_csv('/Users/davidlarre/Desktop/PROYECTOS/ENTREGABLE 1/bmw_original.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/davidlarre/Desktop/PROYECTOS/ENTREGABLE 1/bmw_original.csv'

In [ ]:
for i in ['fecha_registro','fecha_venta']:
    df_bmw[i]=pd.to_datetime(df_bmw[i])

for i in ['fecha_registro','fecha_venta']:
    df_bmw_original[i]=pd.to_datetime(df_bmw_original[i])

In [ ]:
# Consideramos que las fechas completas no aportan información relevante, por lo que vamos a crear nuevas columnas solo con el año
df_bmw['year_registro'] = df_bmw['fecha_registro'].dt.year
df_bmw['year_venta'] = df_bmw['fecha_venta'].dt.year
df_bmw['edad'] = df_bmw['year_venta'] - df_bmw['year_registro']

df_bmw_original['year_registro'] = df_bmw_original['fecha_registro'].dt.year
df_bmw_original['year_venta'] = df_bmw_original['fecha_venta'].dt.year
df_bmw_original['edad'] = df_bmw_original['year_venta'] - df_bmw_original['year_registro']

In [ ]:
df_bmw_original1 = df_bmw_original.copy()
df_bmw1 = df_bmw.copy()

In [ ]:
tabla_nulos(df_bmw).loc[['edad']]

,NULOS,% NULOS,VALORES UNICOS
edad,2423,50.030973,25


In [ ]:
'''moda_paso_1 = df_bmw.groupby(['modelo', 'tipo_coche', 'potencia'])['edad'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'edad': 'MODA_PASO_1'})'''

"moda_paso_1 = df_bmw.groupby(['modelo', 'tipo_coche', 'potencia'])['edad'].agg(obtener_moda).reset_index()\nmoda_paso_1 = moda_paso_1.rename(columns={'edad': 'MODA_PASO_1'})"

In [ ]:
'''df_bmw = df_bmw.merge(moda_paso_1, on=['modelo', 'tipo_coche', 'potencia'], how='left')
df_bmw['edad'] = df_bmw['edad'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)'''

"df_bmw = df_bmw.merge(moda_paso_1, on=['modelo', 'tipo_coche', 'potencia'], how='left')\ndf_bmw['edad'] = df_bmw['edad'].fillna(df_bmw['MODA_PASO_1'])\ndf_bmw.drop(columns=['MODA_PASO_1'], inplace=True)"

In [ ]:
tabla_nulos(df_bmw).loc[['edad']]

,NULOS,% NULOS,VALORES UNICOS
edad,2423,50.030973,25


In [ ]:
'''moda_paso_2 = df_bmw.groupby(['modelo', 'tipo_coche'])['edad'].agg(obtener_moda).reset_index()
moda_paso_2 = moda_paso_2.rename(columns={'edad': 'MODA_PASO_2'})'''

"moda_paso_2 = df_bmw.groupby(['modelo', 'tipo_coche'])['edad'].agg(obtener_moda).reset_index()\nmoda_paso_2 = moda_paso_2.rename(columns={'edad': 'MODA_PASO_2'})"

In [ ]:
'''df_bmw = df_bmw.merge(moda_paso_2, on=['modelo', 'tipo_coche'], how='left')
df_bmw['edad'] = df_bmw['edad'].fillna(df_bmw['MODA_PASO_2'])
df_bmw.drop(columns=['MODA_PASO_2'], inplace=True)'''

"df_bmw = df_bmw.merge(moda_paso_2, on=['modelo', 'tipo_coche'], how='left')\ndf_bmw['edad'] = df_bmw['edad'].fillna(df_bmw['MODA_PASO_2'])\ndf_bmw.drop(columns=['MODA_PASO_2'], inplace=True)"

In [ ]:
tabla_nulos(df_bmw).loc[['edad']]

,NULOS,% NULOS,VALORES UNICOS
edad,2423,50.030973,25


In [ ]:
'''moda_paso_22 = df_bmw.groupby(['modelo', 'potencia'])['edad'].agg(obtener_moda).reset_index()
moda_paso_22 = moda_paso_22.rename(columns={'edad': 'MODA_PASO_22'})'''

"moda_paso_22 = df_bmw.groupby(['modelo', 'potencia'])['edad'].agg(obtener_moda).reset_index()\nmoda_paso_22 = moda_paso_22.rename(columns={'edad': 'MODA_PASO_22'})"

In [ ]:
'''df_bmw = df_bmw.merge(moda_paso_22, on=['modelo', 'potencia'], how='left')
df_bmw['edad'] = df_bmw['edad'].fillna(df_bmw['MODA_PASO_22'])
df_bmw.drop(columns=['MODA_PASO_22'], inplace=True)'''

"df_bmw = df_bmw.merge(moda_paso_22, on=['modelo', 'potencia'], how='left')\ndf_bmw['edad'] = df_bmw['edad'].fillna(df_bmw['MODA_PASO_22'])\ndf_bmw.drop(columns=['MODA_PASO_22'], inplace=True)"

In [ ]:
tabla_nulos(df_bmw).loc[['edad']]

,NULOS,% NULOS,VALORES UNICOS
edad,2423,50.030973,25


In [ ]:
'''moda_paso_3 = df_bmw.groupby(['modelo'])['edad'].agg(obtener_moda).reset_index()
moda_paso_3 = moda_paso_3.rename(columns={'edad': 'MODA_PASO_3'})'''

"moda_paso_3 = df_bmw.groupby(['modelo'])['edad'].agg(obtener_moda).reset_index()\nmoda_paso_3 = moda_paso_3.rename(columns={'edad': 'MODA_PASO_3'})"

In [ ]:
'''df_bmw = df_bmw.merge(moda_paso_3, on=['modelo'], how='left')
df_bmw['edad'] = df_bmw['edad'].fillna(df_bmw['MODA_PASO_3'])
df_bmw.drop(columns=['MODA_PASO_3'], inplace=True)'''

"df_bmw = df_bmw.merge(moda_paso_3, on=['modelo'], how='left')\ndf_bmw['edad'] = df_bmw['edad'].fillna(df_bmw['MODA_PASO_3'])\ndf_bmw.drop(columns=['MODA_PASO_3'], inplace=True)"

In [ ]:
tabla_nulos(df_bmw).loc[['edad']]

,NULOS,% NULOS,VALORES UNICOS
edad,2423,50.030973,25


In [ ]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = df_bmw['fecha_registro'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'edad'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'edad'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 0 de 2423 imputaciones
Porcentaje de acierto: 0.00%


### VERSIÓN 2 - 83,37%

In [ ]:
tabla_nulos(df_bmw).loc[['edad']]

,NULOS,% NULOS,VALORES UNICOS
edad,2423,50.030973,25


In [ ]:
df_bmw['edad'].value_counts()

edad
 5.0     757
 4.0     635
 6.0     435
 3.0     162
 7.0     116
 8.0      51
 2.0      46
 10.0     43
 9.0      39
 12.0     29
 11.0     29
 13.0     28
 14.0     10
 17.0      8
 15.0      8
 1.0       5
 16.0      4
 21.0      4
 18.0      3
 19.0      3
-1.0       1
-5.0       1
 23.0      1
 28.0      1
 24.0      1
Name: count, dtype: int64

In [ ]:
# Definir los rangos de edad y los nuevos valores
rangos = [0, 3, 6, 10, 15, 20, 25, 30]
nuevos_valores = ['1-3', '4-6', '7-10', '11-15', '16-20', '21-25', '>25']

# Aplicar la transformación con pd.cut
df_bmw['edad'] = pd.cut(df_bmw['edad'], bins=rangos, labels=nuevos_valores, right=True)
df_bmw_original['edad'] = pd.cut(df_bmw_original['edad'], bins=rangos, labels=nuevos_valores, right=True)

In [ ]:
'''def get_clase_edad_coche(edad):
  if pd.isna(edad):
    return edad
  elif edad in range(1,4):
    return '1-3'
  elif edad in range(4,7):
    return '4-6'
  elif edad in range(7,11):
    return '7-10'
  elif edad in range(11,16):
    return '11-15'
  elif edad in range(16,21):
    return '16-20'
  elif edad in range(21,26):
    return '21-25'
  else:
    return '>25'''

"def get_clase_edad_coche(edad):\n  if pd.isna(edad):\n    return edad\n  elif edad in range(1,4):\n    return '1-3'\n  elif edad in range(4,7):\n    return '4-6'\n  elif edad in range(7,11):\n    return '7-10'\n  elif edad in range(11,16):\n    return '11-15'\n  elif edad in range(16,21):\n    return '16-20'\n  elif edad in range(21,26):\n    return '21-25'\n  else:\n    return '>25"

In [ ]:
'''# redefiniendo edad_coche como la clase de edad del coche
df_bmw['edad'] = df_bmw['edad'].map(get_clase_edad_coche)
df_bmw_original['edad'] = df_bmw_original['edad'].map(get_clase_edad_coche)'''

"# redefiniendo edad_coche como la clase de edad del coche\ndf_bmw['edad'] = df_bmw['edad'].map(get_clase_edad_coche)\ndf_bmw_original['edad'] = df_bmw_original['edad'].map(get_clase_edad_coche)"

In [ ]:
# guardemos los valores de la moda para edad_coche en tablas diferentes sugun el nivel de agrupación
tabla_moda_edad_coche = df_bmw[~(df_bmw['edad'].isnull())].groupby(['modelo','tipo_coche','potencia'])['edad'].agg([stat.mode])

In [ ]:
tabla_moda_edad_coche_por_modelo_y_tipo_coche = df_bmw[~(df_bmw['edad'].isnull())].groupby(['modelo','tipo_coche'])['edad'].agg([stat.mode])

In [ ]:
tabla_moda_edad_coche_por_modelo = df_bmw[~(df_bmw['edad'].isnull())].groupby(['km'])['edad'].agg([stat.mode])

In [ ]:
# Reemplazemos los nulos en edad_coche con la moda por modelo, tipo_coche y potencia. Cuando no es posible usemos la moda solo por modelo y tipo_coche. Si tampoco esta existe
# usemos la moda por modelo, e si tampoco esta existe le ponemos la moda de edad_coche por el dataset osea '4-6'

for i in df_bmw[df_bmw['edad'].isnull()].index:
  modelo = df_bmw.loc[i,'modelo']
  tipo_coche = df_bmw.loc[i,'tipo_coche']
  potencia = df_bmw.loc[i,'potencia']
 # existe la moda por modelo, tipo_coche y potencia?
  if (modelo,tipo_coche,potencia) in tabla_moda_edad_coche.index:
    edad = tabla_moda_edad_coche.loc[(modelo,tipo_coche,potencia)].values[0]
 # si no, existe la moda solo por modelo y tipo_coche?
  elif (modelo,tipo_coche) in tabla_moda_edad_coche_por_modelo_y_tipo_coche.index:
    edad = tabla_moda_edad_coche_por_modelo_y_tipo_coche.loc[(modelo,tipo_coche)].values[0]
 # si no, existe la moda solo por modelo?
  elif (modelo) in tabla_moda_edad_coche_por_modelo.index:
    edad = tabla_moda_edad_coche_por_modelo.loc[(modelo)].values[0]
  else:
    edad = df_bmw['edad'].mode().values[0]
  df_bmw.loc[i,'edad'] = edad

In [ ]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = df_bmw['fecha_registro'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'edad'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'edad'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 2015 de 2423 imputaciones
Porcentaje de acierto: 83.16%


In [ ]:
df_comparacion = pd.DataFrame({
    'modelo': df_bmw['modelo'],
    'edad_bmw': df_bmw['edad'],
    'edad_original': df_bmw_original['edad']
})

df_comparacion.head(15)

,modelo,edad_bmw,edad_original
0,118,4-6,4-6
1,M4,1-3,1-3
2,320,4-6,4-6
3,420,4-6,4-6
4,425,4-6,4-6
5,335,7-10,7-10
6,325,7-10,7-10
7,118,7-10,7-10
8,Z4,11-15,11-15
9,320,7-10,4-6


### VERSIÓN 3 - 80%

In [ ]:
df_bmw_original = df_bmw_original1.copy()
df_bmw = df_bmw1.copy()

In [ ]:
df_antes = df_bmw.copy()

In [ ]:
df_bmw['edad'].value_counts()

edad
 5.0     757
 4.0     635
 6.0     435
 3.0     162
 7.0     116
 8.0      51
 2.0      46
 10.0     43
 9.0      39
 12.0     29
 11.0     29
 13.0     28
 14.0     10
 17.0      8
 15.0      8
 1.0       5
 16.0      4
 21.0      4
 18.0      3
 19.0      3
-1.0       1
-5.0       1
 23.0      1
 28.0      1
 24.0      1
Name: count, dtype: int64

In [ ]:
# BORRAR NEGATIVOS

In [ ]:
moda_paso_1 = df_bmw.groupby(['modelo', 'potencia', 'tipo_coche', 'bluetooth'])['edad'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'edad': 'MODA_PASO_1'})

In [ ]:
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo', 'potencia', 'tipo_coche','bluetooth'], how='left')
df_bmw['edad'] = df_bmw['edad'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [ ]:
tabla_nulos(df_bmw).loc[['edad']]

,NULOS,% NULOS,VALORES UNICOS
edad,532,10.984927,25


In [ ]:
moda_paso_2 = df_bmw.groupby(['modelo', 'tipo_coche'])['edad'].agg(obtener_moda).reset_index()
moda_paso_2 = moda_paso_2.rename(columns={'edad': 'MODA_PASO_2'})

In [ ]:
df_bmw = df_bmw.merge(moda_paso_2, on=['modelo', 'tipo_coche'], how='left')
df_bmw['edad'] = df_bmw['edad'].fillna(df_bmw['MODA_PASO_2'])
df_bmw.drop(columns=['MODA_PASO_2'], inplace=True)

In [ ]:
tabla_nulos(df_bmw).loc[['edad']]

,NULOS,% NULOS,VALORES UNICOS
edad,42,0.867231,25


In [ ]:
moda_paso_3 = df_bmw.groupby(['modelo'])['edad'].agg(obtener_moda).reset_index()
moda_paso_3 = moda_paso_3.rename(columns={'edad': 'MODA_PASO_3'})

In [ ]:
df_bmw = df_bmw.merge(moda_paso_3, on=['modelo'], how='left')
df_bmw['edad'] = df_bmw['edad'].fillna(df_bmw['MODA_PASO_3'])
df_bmw.drop(columns=['MODA_PASO_3'], inplace=True)

In [ ]:
tabla_nulos(df_bmw).loc[['edad']]

,NULOS,% NULOS,VALORES UNICOS
edad,20,0.412967,25


In [ ]:
moda_paso_4 = df_bmw.groupby(['tipo_coche'])['edad'].agg(obtener_moda).reset_index()
moda_paso_4 = moda_paso_4.rename(columns={'edad': 'MODA_PASO_4'})

In [ ]:
df_bmw = df_bmw.merge(moda_paso_4, on=['tipo_coche'], how='left')
df_bmw['edad'] = df_bmw['edad'].fillna(df_bmw['MODA_PASO_4'])
df_bmw.drop(columns=['MODA_PASO_4'], inplace=True)

In [ ]:
tabla_nulos(df_bmw).loc[['edad']]

,NULOS,% NULOS,VALORES UNICOS
edad,2,0.041297,25


In [ ]:
# Definir los rangos de edad y los nuevos valores
rangos = [0, 3, 6, 10, 15, 20, 25, 30]
nuevos_valores = ['1-3', '4-6', '7-10', '11-15', '16-20', '21-25', '>25']

# Aplicar la transformación con pd.cut
df_bmw['edad'] = pd.cut(df_bmw['edad'], bins=rangos, labels=nuevos_valores, right=True)
df_bmw_original['edad'] = pd.cut(df_bmw_original['edad'], bins=rangos, labels=nuevos_valores, right=True)

In [ ]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = df_bmw['fecha_registro'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'edad'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'edad'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 1944 de 2423 imputaciones
Porcentaje de acierto: 80.23%


### VERSIÓN 4 

In [ ]:
df_bmw_original = df_bmw_original1.copy()
df_bmw = df_bmw1.copy()

In [ ]:
df_antes = df_bmw.copy()

In [ ]:
tabla_nulos(df_bmw).loc[['edad']]

,NULOS,% NULOS,VALORES UNICOS
edad,2423,50.030973,25


In [ ]:
df_bmw['edad'].value_counts()

edad
 5.0     757
 4.0     635
 6.0     435
 3.0     162
 7.0     116
 8.0      51
 2.0      46
 10.0     43
 9.0      39
 12.0     29
 11.0     29
 13.0     28
 14.0     10
 17.0      8
 15.0      8
 1.0       5
 16.0      4
 21.0      4
 18.0      3
 19.0      3
-1.0       1
-5.0       1
 23.0      1
 28.0      1
 24.0      1
Name: count, dtype: int64

In [ ]:
# Definir los rangos de edad y los nuevos valores
rangos = [0, 3, 6, 10, 15, 20, 25, 30]
nuevos_valores = ['1-3', '4-6', '7-10', '11-15', '16-20', '21-25', '>25']

# Aplicar la transformación con pd.cut
df_bmw['edad'] = pd.cut(df_bmw['edad'], bins=rangos, labels=nuevos_valores, right=True)
df_bmw_original['edad'] = pd.cut(df_bmw_original['edad'], bins=rangos, labels=nuevos_valores, right=True)

In [ ]:
'''def get_clase_edad_coche(edad):
  if pd.isna(edad):
    return edad
  elif edad in range(1,4):
    return '1-3'
  elif edad in range(4,7):
    return '4-6'
  elif edad in range(7,11):
    return '7-10'
  elif edad in range(11,16):
    return '11-15'
  elif edad in range(16,21):
    return '16-20'
  elif edad in range(21,26):
    return '21-25'
  else:
    return '>25'''

"def get_clase_edad_coche(edad):\n  if pd.isna(edad):\n    return edad\n  elif edad in range(1,4):\n    return '1-3'\n  elif edad in range(4,7):\n    return '4-6'\n  elif edad in range(7,11):\n    return '7-10'\n  elif edad in range(11,16):\n    return '11-15'\n  elif edad in range(16,21):\n    return '16-20'\n  elif edad in range(21,26):\n    return '21-25'\n  else:\n    return '>25"

In [ ]:
'''# redefiniendo edad_coche como la clase de edad del coche
df_bmw['edad'] = df_bmw['edad'].map(get_clase_edad_coche)
df_bmw_original['edad'] = df_bmw_original['edad'].map(get_clase_edad_coche)'''

"# redefiniendo edad_coche como la clase de edad del coche\ndf_bmw['edad'] = df_bmw['edad'].map(get_clase_edad_coche)\ndf_bmw_original['edad'] = df_bmw_original['edad'].map(get_clase_edad_coche)"

In [ ]:
# guardemos los valores de la moda para edad_coche en tablas diferentes sugun el nivel de agrupación
tabla_moda_edad_coche = df_bmw[~(df_bmw['edad'].isnull())].groupby(['modelo','tipo_coche','potencia'])['edad'].agg([stat.mode])

In [ ]:
tabla_moda_edad_coche_por_modelo_y_tipo_coche = df_bmw[~(df_bmw['edad'].isnull())].groupby(['modelo','tipo_coche'])['edad'].agg([stat.mode])

In [ ]:
tabla_moda_edad_coche_por_modelo = df_bmw[~(df_bmw['edad'].isnull())].groupby(['km'])['edad'].agg([stat.mode])

In [ ]:
# Reemplazemos los nulos en edad_coche con la moda por modelo, tipo_coche y potencia. Cuando no es posible usemos la moda solo por modelo y tipo_coche. Si tampoco esta existe
# usemos la moda por modelo, e si tampoco esta existe le ponemos la moda de edad_coche por el dataset osea '4-6'

for i in df_bmw[df_bmw['edad'].isnull()].index:
  modelo = df_bmw.loc[i,'modelo']
  tipo_coche = df_bmw.loc[i,'tipo_coche']
  potencia = df_bmw.loc[i,'potencia']
 # existe la moda por modelo, tipo_coche y potencia?
  if (modelo,tipo_coche,potencia) in tabla_moda_edad_coche.index:
    edad = tabla_moda_edad_coche.loc[(modelo,tipo_coche,potencia)].values[0]
 # si no, existe la moda solo por modelo y tipo_coche?
  elif (modelo,tipo_coche) in tabla_moda_edad_coche_por_modelo_y_tipo_coche.index:
    edad = tabla_moda_edad_coche_por_modelo_y_tipo_coche.loc[(modelo,tipo_coche)].values[0]
 # si no, existe la moda solo por modelo?
  elif (modelo) in tabla_moda_edad_coche_por_modelo.index:
    edad = tabla_moda_edad_coche_por_modelo.loc[(modelo)].values[0]
  else:
    edad = df_bmw['edad'].mode().values[0]
  df_bmw.loc[i,'edad'] = edad

In [ ]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = df_bmw['fecha_registro'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'edad'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'edad'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 2015 de 2423 imputaciones
Porcentaje de acierto: 83.16%


In [ ]:
df_comparacion = pd.DataFrame({
    'modelo': df_bmw['modelo'],
    'edad_bmw': df_bmw['edad'],
    'edad_original': df_bmw_original['edad']
})

df_comparacion.head(15)

,modelo,edad_bmw,edad_original
0,118,4-6,4-6
1,M4,1-3,1-3
2,320,4-6,4-6
3,420,4-6,4-6
4,425,4-6,4-6
5,335,7-10,7-10
6,325,7-10,7-10
7,118,7-10,7-10
8,Z4,11-15,11-15
9,320,7-10,4-6


## COMPROBACIÓN DE LA IMPUTACIÓN AIRE ACONDICIONADO

### VERSIÓN 1 - 74,69%

In [ ]:
tabla_moda_aire_acondicionado = df_bmw[~(df_bmw['aire_acondicionado'].isnull())].groupby(['edad','tipo_coche','modelo','potencia'])['aire_acondicionado'].agg([stat.mode])

In [ ]:
tabla_moda_aire_acondicionado_por_edad_coche_y_tipo_coche = df_bmw[~(df_bmw['aire_acondicionado'].isnull())].groupby(['edad','tipo_coche'])['aire_acondicionado'].agg([stat.mode])

In [ ]:
# Reemplazemos los nulos en aire_acondicionado con la moda por edad_coche, tipo_coche, modelo y potencia.
# Cuando no es posible uso la moda solo por edad_coche y tipo_coche.
for i in df_bmw[df_bmw['aire_acondicionado'].isnull()].index:
  edad_coche = df_bmw.loc[i,'edad']
  tipo_coche = df_bmw.loc[i,'tipo_coche']
  modelo = df_bmw.loc[i,'modelo']
  potencia = df_bmw.loc[i,'potencia']
 # existe la moda por edad_coche, tipo_coche, modelo y potencia?
  if (edad_coche,tipo_coche,modelo,potencia) in tabla_moda_aire_acondicionado.index:
    aire_acondicionado = tabla_moda_aire_acondicionado.loc[(edad_coche,tipo_coche,modelo,potencia)].values[0]
 # si no, existe la moda solo por edad_coche y tipo_coche?
  elif (edad_coche,tipo_coche) in tabla_moda_aire_acondicionado_por_edad_coche_y_tipo_coche.index:
    aire_acondicionado = tabla_moda_aire_acondicionado_por_edad_coche_y_tipo_coche.loc[(edad_coche,tipo_coche)].values[0]

  df_bmw.loc[i,'aire_acondicionado'] = aire_acondicionado

In [ ]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = bmw['aire_acondicionado'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'aire_acondicionado'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'aire_acondicionado'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 368 de 486 imputaciones
Porcentaje de acierto: 75.72%


### VERSIÓN 2 - 81, 07% modelo', 'edad / 'moddelo', potencia / modelo / moda general

In [ ]:
df_bmw_original = df_bmw_original1.copy()
df_bmw = df_bmw1.copy()

In [ ]:
# Definir los rangos de edad y los nuevos valores
rangos = [0, 3, 6, 10, 15, 20, 25, 30]
nuevos_valores = ['1-3', '4-6', '7-10', '11-15', '16-20', '21-25', '>25']

# Aplicar la transformación con pd.cut
df_bmw['edad'] = pd.cut(df_bmw['edad'], bins=rangos, labels=nuevos_valores, right=True)

In [ ]:
df_bmw

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,...,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta,year_registro,year_venta,edad_coche,color_estandar,edad
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,coupe,True,True,...,NaN,True,NaN,11300.0,2018-01-01,2012.0,2018.0,6.0,1.0,4-6
1,BMW,M4,13929.0,317.0,NaT,petrol,grey,convertible,True,True,...,True,True,True,69700.0,2018-02-01,NaN,2018.0,NaN,1.0,NaN
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,sedan,False,False,...,False,True,False,10200.0,2018-02-01,2012.0,2018.0,6.0,1.0,4-6
3,BMW,420,128035.0,135.0,NaT,diesel,red,convertible,True,True,...,True,True,NaN,25100.0,2018-02-01,NaN,2018.0,NaN,0.0,NaN
4,BMW,425,97097.0,160.0,NaT,diesel,silver,coupe,True,True,...,True,True,True,33400.0,2018-04-01,NaN,2018.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4838,BMW,218 Gran Tourer,39743.0,110.0,NaT,diesel,black,van,False,True,...,False,True,False,14600.0,2018-08-01,NaN,2018.0,NaN,1.0,NaN
4839,BMW,218 Active Tourer,49832.0,100.0,2015-06-01,diesel,grey,van,False,True,...,False,True,True,17500.0,2018-08-01,2015.0,2018.0,3.0,1.0,1-3
4840,BMW,218 Gran Tourer,19633.0,110.0,2015-10-01,diesel,grey,van,False,True,...,False,True,True,17000.0,2018-09-01,2015.0,2018.0,3.0,1.0,1-3
4841,BMW,218 Active Tourer,27920.0,110.0,2016-04-01,diesel,brown,van,True,True,...,False,True,True,22700.0,2018-09-01,2016.0,2018.0,2.0,0.0,1-3


In [ ]:
tabla_nulos(df_bmw).loc[['aire_acondicionado']]

,NULOS,% NULOS,VALORES UNICOS
aire_acondicionado,486,10.035102,2


In [ ]:
moda_paso_1 = df_bmw.groupby(['modelo','edad'])['aire_acondicionado'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'aire_acondicionado': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo', 'edad'], how='left')
df_bmw['aire_acondicionado'] = df_bmw['aire_acondicionado'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [ ]:
tabla_nulos(df_bmw).loc[['aire_acondicionado']]

,NULOS,% NULOS,VALORES UNICOS
aire_acondicionado,236,4.873013,2


In [ ]:
moda_paso_1 = df_bmw.groupby(['modelo','potencia'])['aire_acondicionado'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'aire_acondicionado': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo','potencia'], how='left')
df_bmw['aire_acondicionado'] = df_bmw['aire_acondicionado'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [ ]:
tabla_nulos(df_bmw).loc[['aire_acondicionado']]

,NULOS,% NULOS,VALORES UNICOS
aire_acondicionado,5,0.103242,2


In [ ]:
moda_paso_1 = df_bmw.groupby(['modelo'])['aire_acondicionado'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'aire_acondicionado': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo'], how='left')
df_bmw['aire_acondicionado'] = df_bmw['aire_acondicionado'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [ ]:
tabla_nulos(df_bmw).loc[['aire_acondicionado']]

,NULOS,% NULOS,VALORES UNICOS
aire_acondicionado,2,0.041297,2


In [ ]:
# Calcular la moda de la columna 'bluetooth'
moda_bluetooth = df_bmw['aire_acondicionado'].mode().iloc[0]

# Rellenar los valores nulos con la moda
df_bmw['aire_acondicionado'] = df_bmw['aire_acondicionado'].fillna(moda_bluetooth)
tabla_nulos(df_bmw).loc[['aire_acondicionado']]

,NULOS,% NULOS,VALORES UNICOS
aire_acondicionado,0,0.0,2


In [ ]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = bmw['aire_acondicionado'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'aire_acondicionado'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'aire_acondicionado'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 394 de 486 imputaciones
Porcentaje de acierto: 81.07%


## COMPROBACIÓN DE LA IMPUTACIÓN BLUETOOTH

### VERSIÓN 1 - 75,41%

In [ ]:
df_bmw.head()

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,...,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta,year_registro,year_venta,edad_coche,color_estandar,edad
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,coupe,True,True,...,NaN,True,NaN,11300.0,2018-01-01,2012.0,2018.0,6.0,1.0,4-6
1,BMW,M4,13929.0,317.0,NaT,petrol,grey,convertible,True,True,...,True,True,True,69700.0,2018-02-01,NaN,2018.0,NaN,1.0,NaN
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,sedan,False,False,...,False,True,False,10200.0,2018-02-01,2012.0,2018.0,6.0,1.0,4-6
3,BMW,420,128035.0,135.0,NaT,diesel,red,convertible,True,True,...,True,True,NaN,25100.0,2018-02-01,NaN,2018.0,NaN,0.0,NaN
4,BMW,425,97097.0,160.0,NaT,diesel,silver,coupe,True,True,...,True,True,True,33400.0,2018-04-01,NaN,2018.0,NaN,0.0,NaN


In [ ]:
tabla_moda_bluetooth = df_bmw[~(df_bmw['bluetooth'].isnull())].groupby(['edad','tipo_coche','modelo','potencia'])['bluetooth'].agg([stat.mode])

In [ ]:
tabla_moda_bluetooth_por_edad_coche_y_tipo_coche = df_bmw[~(df_bmw['bluetooth'].isnull())].groupby(['edad','tipo_coche'])['bluetooth'].agg([stat.mode])

In [ ]:
# Reemplazemos los nulos en bluetooth con la moda por edad_coche, tipo_coche, modelo y potencia. Cuando no es posible usemos la moda solo por edad_coche y tipo_coche. Si tampoco esta existe
# usemos la moda del dateset.
for i in df_bmw[df_bmw['bluetooth'].isnull()].index:
  edad_coche = df_bmw.loc[i,'edad']
  tipo_coche = df_bmw.loc[i,'tipo_coche']
  modelo = df_bmw.loc[i,'modelo']
  potencia = df_bmw.loc[i,'potencia']
 # existe la moda por edad_coche, tipo_coche, modelo y potencia?
  if (edad_coche,tipo_coche,modelo,potencia) in tabla_moda_bluetooth.index:
    bluetooth = tabla_moda_bluetooth.loc[(edad_coche,tipo_coche,modelo,potencia)].values[0]
 # si no, existe la moda solo por edad_coche y tipo_coche?
  elif (edad_coche,tipo_coche) in tabla_moda_bluetooth_por_edad_coche_y_tipo_coche.index:
    bluetooth = tabla_moda_bluetooth_por_edad_coche_y_tipo_coche.loc[(edad_coche,tipo_coche)].values[0]
  else:
    bluetooth = df_bmw['bluetooth'].mode().values[0]
  df_bmw.loc[i,'bluetooth'] = bluetooth

In [ ]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = bmw['bluetooth'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'bluetooth'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'bluetooth'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 532 de 728 imputaciones
Porcentaje de acierto: 73.08%


### VERSIÓN 2 - 79,26%

In [ ]:
df_bmw

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,...,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta,year_registro,year_venta,edad_coche,color_estandar,edad
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,coupe,True,True,...,True,True,NaN,11300.0,2018-01-01,2012.0,2018.0,6.0,1.0,4-6
1,BMW,M4,13929.0,317.0,NaT,petrol,grey,convertible,True,True,...,True,True,True,69700.0,2018-02-01,NaN,2018.0,NaN,1.0,NaN
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,sedan,False,False,...,False,True,False,10200.0,2018-02-01,2012.0,2018.0,6.0,1.0,4-6
3,BMW,420,128035.0,135.0,NaT,diesel,red,convertible,True,True,...,True,True,NaN,25100.0,2018-02-01,NaN,2018.0,NaN,0.0,NaN
4,BMW,425,97097.0,160.0,NaT,diesel,silver,coupe,True,True,...,True,True,True,33400.0,2018-04-01,NaN,2018.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4838,BMW,218 Gran Tourer,39743.0,110.0,NaT,diesel,black,van,False,True,...,False,True,False,14600.0,2018-08-01,NaN,2018.0,NaN,1.0,NaN
4839,BMW,218 Active Tourer,49832.0,100.0,2015-06-01,diesel,grey,van,False,True,...,False,True,True,17500.0,2018-08-01,2015.0,2018.0,3.0,1.0,1-3
4840,BMW,218 Gran Tourer,19633.0,110.0,2015-10-01,diesel,grey,van,False,True,...,False,True,True,17000.0,2018-09-01,2015.0,2018.0,3.0,1.0,1-3
4841,BMW,218 Active Tourer,27920.0,110.0,2016-04-01,diesel,brown,van,True,True,...,False,True,True,22700.0,2018-09-01,2016.0,2018.0,2.0,0.0,1-3


In [ ]:
df_bmw_original = df_bmw_original1.copy()
df_bmw = df_bmw1.copy()

In [ ]:
moda_paso_1 = df_bmw.groupby(['edad','tipo_coche','modelo','potencia', 'asientos_traseros_plegables', 'aire_acondicionado'])['bluetooth'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'bluetooth': 'MODA_PASO_1'})

df_bmw = df_bmw.merge(moda_paso_1, on=['edad','tipo_coche','modelo','potencia', 'asientos_traseros_plegables', 'aire_acondicionado'], how='left')
df_bmw['bluetooth'] = df_bmw['bluetooth'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [ ]:
tabla_nulos(df_bmw).loc[['bluetooth']]

,NULOS,% NULOS,VALORES UNICOS
bluetooth,663,13.689862,2


In [ ]:
moda_paso_1 = df_bmw.groupby(['modelo', 'asientos_traseros_plegables', 'aire_acondicionado'])['bluetooth'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'bluetooth': 'MODA_PASO_1'})

df_bmw = df_bmw.merge(moda_paso_1, on=['modelo', 'asientos_traseros_plegables', 'aire_acondicionado'], how='left')
df_bmw['bluetooth'] = df_bmw['bluetooth'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [ ]:
tabla_nulos(df_bmw).loc[['bluetooth']]

,NULOS,% NULOS,VALORES UNICOS
bluetooth,523,10.799091,2


In [ ]:
moda_paso_1 = df_bmw.groupby(['modelo','elevalunas_electrico'])['bluetooth'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'bluetooth': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo','elevalunas_electrico'], how='left')
df_bmw['bluetooth'] = df_bmw['bluetooth'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [ ]:
tabla_nulos(df_bmw).loc[['bluetooth']]

,NULOS,% NULOS,VALORES UNICOS
bluetooth,5,0.103242,2


In [ ]:
moda_paso_1 = df_bmw.groupby(['modelo'])['bluetooth'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'bluetooth': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo'], how='left')
df_bmw['bluetooth'] = df_bmw['bluetooth'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [ ]:
tabla_nulos(df_bmw).loc[['bluetooth']]

,NULOS,% NULOS,VALORES UNICOS
bluetooth,2,0.041297,2


In [ ]:
# Calcular la moda de la columna 'bluetooth'
moda_bluetooth = df_bmw['bluetooth'].mode().iloc[0]

# Rellenar los valores nulos con la moda
df_bmw['bluetooth'] = df_bmw['bluetooth'].fillna(moda_bluetooth)

In [ ]:
tabla_nulos(df_bmw).loc[['bluetooth']]

,NULOS,% NULOS,VALORES UNICOS
bluetooth,0,0.0,2


In [ ]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = bmw['bluetooth'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'bluetooth'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'bluetooth'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 577 de 728 imputaciones
Porcentaje de acierto: 79.26%


In [42]:
# Ver que distribucion de valores tiene la columna bluetooth
df_bmw['bluetooth'].value_counts(normalize=True)

bluetooth
False    0.757959
True     0.242041
Name: proportion, dtype: float64

## COMPROBACIÓN DE LA IMPUTACIÓN ALERTA LIMITE VELOCIDAD

### VERSIÓN 1 - 65,38%

In [ ]:
tabla_nulos(df_bmw).loc[['alerta_lim_velocidad']]

,NULOS,% NULOS,VALORES UNICOS
alerta_lim_velocidad,728,15.032005,2


In [ ]:
distribuciones['alerta_lim_velocidad']

alerta_lim_velocidad
True     0.54435
False    0.45565
Name: proportion, dtype: float64

In [ ]:
tabla_moda_alerta_lim_velocidad = df_bmw[~(df_bmw['alerta_lim_velocidad'].isnull())].groupby(['edad','tipo_coche','modelo','potencia','volante_regulable','elevalunas_electrico'])['alerta_lim_velocidad'].agg([stat.mode])

In [ ]:
tabla_nulos(df_bmw).loc[['alerta_lim_velocidad']]

,NULOS,% NULOS,VALORES UNICOS
alerta_lim_velocidad,728,15.032005,2


In [ ]:
tabla_moda_alerta_lim_velocidad_mas_general = df_bmw[~(df_bmw['alerta_lim_velocidad'].isnull())].groupby(['edad','tipo_coche','volante_regulable','elevalunas_electrico'])['alerta_lim_velocidad'].agg([stat.mode])

In [ ]:
tabla_nulos(df_bmw).loc[['alerta_lim_velocidad']]

,NULOS,% NULOS,VALORES UNICOS
alerta_lim_velocidad,728,15.032005,2


In [ ]:
# reemplazemos los nulos en alerta_lim_velocidad con la moda por 'edad_coche', 'tipo_coche','modelo','potencia','volante_regulable' y 'elevalunas_electrico'.
# Cuando no es posible usemos la moda por 'edad_coche','tipo_coche','volante_regulable', y 'elevalunas_electrico'

for i in df_bmw[df_bmw['alerta_lim_velocidad'].isnull()].index:
  edad_coche = df_bmw.loc[i,'edad']
  tipo_coche = df_bmw.loc[i,'tipo_coche']
  modelo = df_bmw.loc[i,'modelo']
  potencia = df_bmw.loc[i,'potencia']
  volante_regulable = df_bmw.loc[i,'volante_regulable']
  elevalunas_electrico = df_bmw.loc[i,'elevalunas_electrico']
 # existe la moda por ''edad_coche,'tipo_coche','modelo','potencia','volante_regulable','elevalunas_electrico'?
  if (edad_coche,tipo_coche,modelo,potencia,volante_regulable,elevalunas_electrico) in tabla_moda_alerta_lim_velocidad.index:
    alerta_lim_velocidad = tabla_moda_alerta_lim_velocidad.loc[(edad_coche,tipo_coche,modelo,potencia,volante_regulable,elevalunas_electrico)].values[0]
 # si no, existe la moda solo por 'edad_coche','tipo_coche','volante_regulable', y 'elevalunas_electrico'?
  elif (edad_coche,tipo_coche,volante_regulable,elevalunas_electrico) in tabla_moda_alerta_lim_velocidad_mas_general.index:
    alerta_lim_velocidad = tabla_moda_alerta_lim_velocidad_mas_general.loc[(edad_coche,tipo_coche,volante_regulable,elevalunas_electrico)].values[0]

  df_bmw.loc[i,'alerta_lim_velocidad'] = alerta_lim_velocidad

In [ ]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = bmw['alerta_lim_velocidad'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'alerta_lim_velocidad'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'alerta_lim_velocidad'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 440 de 728 imputaciones
Porcentaje de acierto: 60.44%


### VERSIÓN 2 - 72,53% 'potencia', 'km / modelo', 'potencia / moda general

In [ ]:
df_bmw_original = df_bmw_original1.copy()
df_bmw = df_bmw1.copy()

In [ ]:
tabla_nulos(df_bmw).loc[['alerta_lim_velocidad']]

,NULOS,% NULOS,VALORES UNICOS
alerta_lim_velocidad,728,15.032005,2


In [ ]:
distribuciones['alerta_lim_velocidad']

alerta_lim_velocidad
True     0.54435
False    0.45565
Name: proportion, dtype: float64

In [ ]:
df_bmw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   marca                        3873 non-null   object        
 1   modelo                       4840 non-null   object        
 2   km                           4841 non-null   float64       
 3   potencia                     4842 non-null   float64       
 4   fecha_registro               2420 non-null   datetime64[ns]
 5   tipo_gasolina                4838 non-null   object        
 6   color                        4398 non-null   object        
 7   tipo_coche                   4841 non-null   object        
 8   volante_regulable            4839 non-null   object        
 9   aire_acondicionado           4357 non-null   object        
 10  camara_trasera               4841 non-null   object        
 11  asientos_traseros_plegables  1452 non-null 

In [ ]:
moda_paso_1 = df_bmw.groupby(['potencia', 'km'])['alerta_lim_velocidad'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'alerta_lim_velocidad': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['potencia', 'km'], how='left')
df_bmw['alerta_lim_velocidad'] = df_bmw['alerta_lim_velocidad'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [ ]:
tabla_nulos(df_bmw).loc[['alerta_lim_velocidad']]

,NULOS,% NULOS,VALORES UNICOS
alerta_lim_velocidad,723,14.928763,2


In [ ]:
moda_paso_1 = df_bmw.groupby(['modelo', 'potencia'])['alerta_lim_velocidad'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'alerta_lim_velocidad': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo', 'potencia'], how='left')
df_bmw['alerta_lim_velocidad'] = df_bmw['alerta_lim_velocidad'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [ ]:
tabla_nulos(df_bmw).loc[['alerta_lim_velocidad']]

,NULOS,% NULOS,VALORES UNICOS
alerta_lim_velocidad,13,0.268429,2


In [ ]:
# Calcular la moda de la columna 'bluetooth'
moda_bluetooth = df_bmw['alerta_lim_velocidad'].mode().iloc[0]

# Rellenar los valores nulos con la moda
df_bmw['alerta_lim_velocidad'] = df_bmw['alerta_lim_velocidad'].fillna(moda_bluetooth)
tabla_nulos(df_bmw).loc[['alerta_lim_velocidad']]

,NULOS,% NULOS,VALORES UNICOS
alerta_lim_velocidad,0,0.0,2


In [ ]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = bmw['alerta_lim_velocidad'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'alerta_lim_velocidad'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'alerta_lim_velocidad'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 528 de 728 imputaciones
Porcentaje de acierto: 72.53%
